In [1]:
from fineNeat import loadHyp, updateHyp, load_task, GymTask, games 
from fineNeat import Neat, Ind, viewInd, fig2img 
from matplotlib import pyplot as plt 
from tqdm import tqdm 

game = games["slimevolley"]
task = GymTask(game)

hyp_default = 'fineNeat/fineNeat/p/default_sneat.json'
hyp_adjust = "fineNeat/fineNeat/p/volley.json"
fileName = "volley"

hyp = loadHyp(pFileName=hyp_default, load_task=load_task)
updateHyp(hyp,load_task,hyp_adjust)

hyp["cap_layer"] = 3
neat = Neat(hyp)

#### Population TestBed 

In [6]:
neat = Neat(hyp)

neat.initPop() # fixed naive input-output topology + random weight / on & off 

neat.rankPop() # rank 
neat.speciate() # speciate  
grid_img = neat.printSpecies(neat.species) # visualize 
neat.evolvePop() # fixed

 :: Total of species:  1
 :: Species:  0  :: Offspring:  128


#### Mutation TestBed 

In [30]:
from fineNeat import viewInd, fig2img, Ind
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import numpy as np 
ind = Ind.from_shapes([[12, 10],[10, 10],[10, 3]])
frames = []

for i in tqdm(range(100)): 
    connG, nodeG = np.copy(ind.conn), np.copy(ind.node)
    connG[3,:] = 1
    connG, nodeG, _ = ind.mutSparsity(p=hyp)
    fine_sparse = Ind(connG, nodeG).express()
    connG, nodeG, _ = ind.mutAddNode(connG, nodeG, gen=1, innov=None, p=hyp) # Bug :: connecting mutSparsity with addNode & addConn is Buggy
    fine_addnode = Ind(connG, nodeG).express()
    connG, nodeG, _ = ind.mutAddConn(connG, nodeG, gen=1, innov=None)
    
    child = Ind(connG, nodeG)
    if child.express():
        fig, ax = viewInd(child)
        frames.append(fig2img(fig))
        plt.close(fig)
        ind = child
    else: 
        print("Failed to express child")

frames[0].save("addSparseNodeConn.gif", save_all=True, append_images=frames[1:], duration=100, loop=0) # save frames to gif

100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


#### Population TestBed 

In [8]:
# Test population 
from fineNeat import Ind 
import gym

env = gym.make("SlimeVolley-v0")

population_size = 20
population = [Ind.from_shapes([(game.input_size, 5), (5, game.output_size)]) for _ in range(population_size)]